# Memory Representation in Dialogue Systems (Part 1)

The following notebook is the result of an NLP project that explores the question, "How could interaction be stored in memory, and how can that information be leveraged for further use?"

[Dialogue systems](https://en.wikipedia.org/wiki/Dialog_system) can be quite useful, but have difficulty keeping track of concepts and entities dynamically. To enhance their capabilities, this notebook will provide a simple software implementation of the Dynamic Memory Model, which is intended to test the ability of this model to enhance a dialogue system's ability to understand user input by forming high-level concepts that evolve over time. This notebook will show how such a system can update its internal state based on natural language facts, and retrieve results based on natural language questions.

The code below is written in Python, and uses a [Neo4j Graph Database](https://neo4j.com/product/) to provide non-volatile storage and efficient querying capabilities.

The test corpus is supplied by the [bAbI Tasks Data 1-20 (v1.2)](https://research.fb.com/downloads/babi/). It contains a sequence of English sentences to provide the system knowledge of a simple domain involving characters moving to different rooms.

## Prerequisites to Running this Notebook
- [Python](https://www.python.org/downloads/) (3.5+)
- Python packages (install via pip): `pandas`, `numpy`, `nltk`, `scikit-learn`, `neo4j-driver`
- [Neo4j](https://neo4j.com/download/) (3.1+)

### TODO

- Py2neo visualization in Jupyter notebooks http://nicolewhite.github.io/neo4j-jupyter/hello-world.html

## Process the Text

### Import DataFrames
First we will use `pandas` to import `qa1_single-supporting-fact_train.txt` from our corpus into a DataFrame called `data`. Every line in this document represents one sentence, which will be split using `nltk`'s word tokenizer.

In [1]:
import pandas as pd
import numpy as np
import nltk
from sklearn.metrics import accuracy_score

In [2]:
# Read the bAbI data as CSV
filename = 'resources/qa1_single-supporting-fact_train.txt'
data = pd.read_csv(filename, delimiter='\t', names=['sentence', 'answer', 'factid'])
data = data.fillna('')

# Tag each sentence as a statement or question
tag_sentence = lambda row: 'S' if row.answer == '' else 'Q'
data['type'] = data.apply(tag_sentence, axis=1)

# Use NLTK to tokenize the sentences into arrays of words
tokenize = lambda row: nltk.word_tokenize(row.sentence)[1:]
data.sentence = data.apply(tokenize, axis=1)

Every `sentence` in this frame can either be a factual statement, or a question about the preceeding statements. The `answer` column supplies the answer to a question (it is blank if the sentence is a statement).

In [3]:
data[:6]

,sentence,answer,factid,type
0,"[Mary, moved, to, the, bathroom, .]",,,S
1,"[John, went, to, the, hallway, .]",,,S
2,"[Where, is, Mary, ?]",bathroom,1,Q
3,"[Daniel, went, back, to, the, hallway, .]",,,S
4,"[Sandra, moved, to, the, garden, .]",,,S
5,"[Where, is, Daniel, ?]",hallway,4,Q


The `data` DataFrame is further split up into the `statements` and `questions` DataFrames for easy access to all statements and questions respectively.

In [4]:
# Create a DataFrame with just the statements
def statements():
    return data[data.type == 'S'] \
        .reset_index(drop=True) \
        .drop('answer', axis=1) \
        .drop('factid', axis=1) \
        .drop('type', axis=1)

# Create a DataFrame with just the questions
def questions():
    return data[data.type == 'Q'] \
        .reset_index(drop=True) \
        .drop('type', axis=1)

In [5]:
statements()[:4]

,sentence
0,"[Mary, moved, to, the, bathroom, .]"
1,"[John, went, to, the, hallway, .]"
2,"[Daniel, went, back, to, the, hallway, .]"
3,"[Sandra, moved, to, the, garden, .]"


In [6]:
questions()[:2]

,sentence,answer,factid
0,"[Where, is, Mary, ?]",bathroom,1
1,"[Where, is, Daniel, ?]",hallway,4


### Extract Entities
Next, we will extract the relevant entities from each statement and question so that we can more easily reason with these sentences.

#### POS Tagging
To process each sentence and produce a useful statement or question object, all that is necessary (for this dataset) is to use a part-of-speech tagger.

In [7]:
# Tag each token as a part of speech
pos_tag = lambda row: nltk.pos_tag(row.sentence)
data['tag'] = data.apply(pos_tag, axis=1)

In [8]:
data[['sentence', 'tag']][:5]

,sentence,tag
0,"[Mary, moved, to, the, bathroom, .]","[(Mary, NNP), (moved, VBD), (to, TO), (the, DT..."
1,"[John, went, to, the, hallway, .]","[(John, NNP), (went, VBD), (to, TO), (the, DT)..."
2,"[Where, is, Mary, ?]","[(Where, WRB), (is, VBZ), (Mary, NNP), (?, .)]"
3,"[Daniel, went, back, to, the, hallway, .]","[(Daniel, NNP), (went, VBD), (back, RB), (to, ..."
4,"[Sandra, moved, to, the, garden, .]","[(Sandra, NNP), (moved, VBD), (to, TO), (the, ..."


#### Statements
Due to the simplicity of the data, each statement can be thought of as a `(subject, relation, object)` triple. We would like to define a function `S`, that when given a sequence of statement tokens, produces this triple. For instance,
```
S([Mary, moved, to, the, bathroom, .]) = (Mary, moved, bathroom).
```
This allows one to construct a graph of relationships between objects, as we will see in the next sections. 

We can use the POS tags in the sentence to achieve this. If there is a word tagged as a proper noun, it is the subject, if there's a verb, it is the relation, and if there's a simple noun, it is the object.

In [9]:
def extract_statement(tags):
    '''Extracts a (subject, relation, object) triple from each statement based on the POS tags'''
    subject, relation, obj = '', '', ''
    for word,tag in tags:
        if tag == 'NNP':
            subject = word
        elif tag == 'VBD':
            relation = word
        elif tag == 'NN':
            obj = word
    return (subject, relation, obj)

#### Questions
To test the graph, we would like to define another function `Q`, that when given a sequence of quesiton tokens, returns the entity that the question is asking for.
```
Q([Where, is, Mary, ?]) = Mary
```

The result is the subject we are querying for, whose query should return us a room to answer the question.

In [10]:
def extract_question(tags):
    '''Extracts the entity under discussion from each question based on the POS tags'''
    eud = ''
    for word,tag in tags:
        if tag == 'NNP':
            eud = word
    return eud

Then, call the appropriate function to extract the correct info

In [11]:
def extract(row):
    '''Extracts the appropriate data given a processed DataFrame row'''
    if row.type == 'S':
        return extract_statement(row.tag)
    else: 
        return extract_question(row.tag)

In [12]:
data['extracted'] = data.apply(extract, axis=1)

In [13]:
data[['sentence', 'extracted']][:5]

,sentence,extracted
0,"[Mary, moved, to, the, bathroom, .]","(Mary, moved, bathroom)"
1,"[John, went, to, the, hallway, .]","(John, went, hallway)"
2,"[Where, is, Mary, ?]",Mary
3,"[Daniel, went, back, to, the, hallway, .]","(Daniel, went, hallway)"
4,"[Sandra, moved, to, the, garden, .]","(Sandra, moved, garden)"


Voila, extraction is complete

### Debug Functions

These are handy debugging functions that we will use for evaluation.

This function finds all statements that refer to a person.

In [14]:
def person_statements(person):
    '''Get all statements that refer to the specified person'''
    stat = statements()
    return stat[stat.extracted.map(lambda t: t[0] == person)]

For instance, we can find all statements that refer to Sandra.

In [15]:
person_statements('Sandra')[:3]

,sentence,tag,extracted
3,"[Sandra, moved, to, the, garden, .]","[(Sandra, NNP), (moved, VBD), (to, TO), (the, ...","(Sandra, moved, garden)"
5,"[Sandra, journeyed, to, the, bathroom, .]","[(Sandra, NNP), (journeyed, VBD), (to, TO), (t...","(Sandra, journeyed, bathroom)"
10,"[Sandra, travelled, to, the, office, .]","[(Sandra, NNP), (travelled, VBD), (to, TO), (t...","(Sandra, travelled, office)"


This function finds the `n` most recent statements that refer to a person.

In [16]:
def person_statements_recent(person, n=5):
    '''Get the n most recent statements that refer to the specified person in reverse chronological order'''
    return person_statements(person)[-n:].iloc[::-1]

For instance, we can find the 3 most recent statements Daniel has been referred in.

In [17]:
person_statements_recent('Daniel', n=3)

,sentence,tag,extracted
1999,"[Daniel, went, to, the, garden, .]","[(Daniel, NNP), (went, VBD), (to, TO), (the, D...","(Daniel, went, garden)"
1996,"[Daniel, travelled, to, the, kitchen, .]","[(Daniel, NNP), (travelled, VBD), (to, TO), (t...","(Daniel, travelled, kitchen)"
1992,"[Daniel, moved, to, the, office, .]","[(Daniel, NNP), (moved, VBD), (to, TO), (the, ...","(Daniel, moved, office)"


## Build the Graph
Once we have processed the data into triples, we can build graphs from them. Below we have defined a couple functions to reset the database and run queries. We will use Neo4j's driver to accomplish this. Note that if the URL or auth credentials are different, you will need to change them.

In [18]:
from neo4j.v1 import GraphDatabase, basic_auth

In [19]:
# Create a neo4j session
# NOTE: Make sure that URL/credentials are correct and that Neo4j is running
driver = GraphDatabase.driver('bolt://localhost:7687', auth=basic_auth('neo4j', 'neo4j'))

In [20]:
# WARNING: This will clear the database when run!
def reset_db():
    '''Remove all nodes and relationships from the database'''
    session = driver.session()
    session.run('MATCH (n) DETACH DELETE n')

In [21]:
def create(query, n=0):
    '''Create a graph based on each triple in the extracted statements'''
    session = driver.session()
    stat = statements()
    n = len(stat) if n <= 0 else n
    for subject,relation,obj in stat[:n].extracted:
        session.run(query, { 
            'subject': subject,
            'relation': relation,
            'obj': obj
        })

### V1: Direct relationships
One of the first impulses when building the graph is to represent the subject and object as nodes, and the relations as edges between them.

In [22]:
reset_db()

In [23]:
# Create a direct relationship between subject and object
v1_query = '''
    MERGE (s:SUBJECT {name: $subject}) 
    MERGE (o:OBJECT  {name: $obj}) 
    MERGE (s)-[r:RELATION {name: $relation}]->(o)
'''

create(v1_query)

Run the query below and see what the graph looks like. Pop open a new tab in the Neo4j browser (default http://localhost:7474/browser/) and run the query:
```
MATCH (n) RETURN n LIMIT 50
```
The graph is a reasonable first start, as the relations point each person to where they have been. But this poses a potential problem: how do we know where each person is right now, or where they have been previously? All we can know from the graph is which rooms a person has been in, because they may have visited them all multiple times.

<img src="screenshots/simple-relation.png",width=700>

### V2: Nodes for relationships
One approach is to form a linked list of "events". Each event corresponds to a person updating the room that they are in. Since we chose edges to be our relations, we cannot form edges between relations. To alleviate this, we can transform the relation to a node, and draw two edges to form a 3-node triple.

In [24]:
reset_db()

In [25]:
# Represent each relation as a node
v2_query = '''
    MERGE (s:SUBJECT {name: $subject})
    MERGE (o:OBJECT  {name: $obj})
    CREATE (s)-[:R0]->(r:RELATION {name: $relation})-[:R1]->(o)
'''

create(v2_query)

Run the query again and see what changed. This is better, since we can see how often a room has been visited, but still doesn't solve the question as to which room a person is in at any given time.

### V3: Linked list of relationships
The final step is to build the linked list based on the order in which the relations were created. This will allow us to not only find the room a person is in right now, but produce a list of rooms that they were in, in the order that they were visited.

In [26]:
reset_db()

In [27]:
# Represent each relation as a node, ordered by a linked list (per subject)
v3_query = '''
    MERGE (s:SUBJECT {name: $subject})
    MERGE (o:OBJECT  {name: $obj})
    
    WITH s,o
    
    // Create an new relation between the subject and object
    CREATE (s)-[:R0]->(r:RELATION {name: $relation})-[:R1]->(o)
    CREATE (s)-[h:HEAD]->(r) // Make the newly created relation the head of the list
    
    WITH s,r,o,h
    
    // Find the previous head of the list (if none exist, this query will terminate here)
    MATCH (s)-[h_prev:HEAD]->(r_prev:RELATION)
    WHERE h_prev <> h
    
    // Complete the link, remove the previous head pointer
    CREATE (r_prev)-[:NEXT]->(r)
    DELETE h_prev
'''

In [28]:
session = driver.session()
# Create an index for faster access
session.run('CREATE INDEX ON :SUBJECT(name)')
session.run('CREATE INDEX ON :RELATION(name)')
session.run('CREATE INDEX ON :OBJECT(name)')
create(v3_query)

Check the new graph out and see what changed. It's helpful to change the colors of the nodes and edges to visualize this better.

<img src="screenshots/local-list.png",width=800>

## Query the Graph
Now we can ask the graph useful questions.
#### Find the room a person is in

In [29]:
def find_person(person):
    '''Find the room a person is currently in'''
    query = '''
        MATCH (s:SUBJECT {name:$name})-[:HEAD]->(r:RELATION)-->(o:OBJECT)
        RETURN s AS subject, r AS relation, o AS obj
    '''
    return session.run(query, {'name': person})

Using the graph-querying function above we can ask, "Where is Mary?"

In [30]:
# Note: If this is run less than a second after creating the knowledge graph, 
# the Python driver may cause a race condition where the graph 
# isn't finished updating, which could give you the wrong answer.
session = driver.session()
record = find_person('Mary').single()
print(record['obj'].get('name'))

kitchen


According to the graph, Mary is in the kitchen. Verify that this is true with a debug function.

In [31]:
person_statements_recent('Mary', n=1)

,sentence,tag,extracted
1994,"[Mary, journeyed, to, the, kitchen, .]","[(Mary, NNP), (journeyed, VBD), (to, TO), (the...","(Mary, journeyed, kitchen)"


Now we see that this is indeed true, and we see the corresponding sentence as well.

#### Find the rooms a person has been in (reverse chronological order)

In [32]:
def find_person_history(person, n=100):
    '''Find the list of rooms a person was in, ordered by recency'''
    length = str(n) if n >= 1 else ''
    
    query = '''
        MATCH (s:SUBJECT {name:$name})-[:HEAD]->(r:RELATION)-->(o:OBJECT)
        MATCH (s)-->(r_prev:RELATION)-[k*1..%s]->(r), (r_prev)-->(o_prev:OBJECT)
        
        WITH size(k) AS dist, r, o, r_prev, o_prev
        ORDER BY size(k)
        
        WITH r, o, r_prev, o_prev
        RETURN [r.name] + collect(r_prev.name) AS relation, [o.name] + collect(o_prev.name) AS obj
    '''
    query = query % length
    
    session = driver.session()
    record = session.run(query, {'name': person}).single()
    history = list(zip(record['relation'], record['obj']))[:-1]
    
    return history

A more advanced question that we get for free based on the graph structure is, "Where has John been recently?"

In [33]:
find_person_history('John', n=5)

[('went', 'bedroom'),
 ('went', 'garden'),
 ('went', 'office'),
 ('', 'bedroom'),
 ('travelled', 'hallway')]

Verify that John has been to to those places, in that order.

In [34]:
person_statements_recent('John', n=5)

,sentence,tag,extracted
1995,"[John, went, back, to, the, bedroom, .]","[(John, NNP), (went, VBD), (back, RB), (to, TO...","(John, went, bedroom)"
1989,"[John, went, back, to, the, garden, .]","[(John, NNP), (went, VBD), (back, RB), (to, TO...","(John, went, garden)"
1986,"[John, went, back, to, the, office, .]","[(John, NNP), (went, VBD), (back, RB), (to, TO...","(John, went, office)"
1982,"[John, journeyed, to, the, bedroom, .]","[(John, NNP), (journeyed, NN), (to, TO), (the,...","(John, , bedroom)"
1979,"[John, travelled, to, the, hallway, .]","[(John, NNP), (travelled, VBD), (to, TO), (the...","(John, travelled, hallway)"


Note that some of the triples may be blank. This is because the POS tagger made a mistake when tagging the tokens. This doesn't affect our accuracy.

#### Find the history of visitors for a room

In [35]:
def find_room_visitors(room):
    '''Find the list of visitors a room has, ordered by recency'''
    
    query = '''
        MATCH (r:RELATION)-->(o:OBJECT {name:$name})
        RETURN count(r) AS count
    '''
    
    session = driver.session()
    record = session.run(query, {'name': room}).single()
    
    return record['count']

Just for fun, we can find out how many times a room has been visited. "How many times has the office been visited?"

In [36]:
find_room_visitors('office')

334

## Calculate an Accuracy Score

In [37]:
def get_answers(row):
    '''Given an input row merge the statement in the graph, 
    or query the graph if it is a question'''
    if row.type == 'S':
        subject,relation,obj = row.extracted
        session.run(v3_query, {
            'subject': subject,
            'relation': relation,
            'obj': obj
        })
        return ''
    elif row.type == 'Q':
        person = row.extracted
        # WARNING: do not consume the result (e.g., call .consume() or .single()) 
        # until the entire iteration is done.
        # Failure to do so may cause the queries to be VERY slow!
        return find_person(person)

Start all over, and run through the entire dataset.

In [38]:
reset_db()

In [39]:
session = driver.session()
results = data.apply(get_answers, axis=1)
results = [x for x in results if x != '']
predicted = [result.single()['obj'].get('name') for result in results]

The `predicted` array contains the predicted answer to each question.`

In [40]:
predicted[:5]

['bathroom', 'hallway', 'hallway', 'office', 'bathroom']

The `actual` array contains the actual answers to all questions.

In [41]:
actual = list(data[data.type == 'Q'].answer)

In [42]:
actual[:5]

['bathroom', 'hallway', 'hallway', 'office', 'bathroom']

In [43]:
accuracy_score(actual, predicted)

1.0

And just like that, we get an accuracy of 100%. Of course, this dataset is very simple, but the graph we created can generalize to any statements of the form, `(subject, relation, object)`.